<a href="https://colab.research.google.com/github/erfrits/girls_day_ds/blob/main/nlp_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
import plotly.express as px

# Nyelvi modellek

## Szózsákmodell (bag of words)

### Egyszerű példa

In [5]:
sentences = [
    'macska macska macska',
    'kutya kutya',
    'kutya macska kutya'
]

In [7]:
# szavak a szövegben
words = list({w for s in sentences for w in s.split(" ")})
words

['macska', 'kutya']

In [9]:
# szövegből vektor: hányszor szerepel benne?
vect = [{w: s.count(w) for w in words} for s in sentences]
vect

[{'macska': 3, 'kutya': 0},
 {'macska': 0, 'kutya': 2},
 {'macska': 1, 'kutya': 2}]

In [10]:
pd.DataFrame(vect, index=sentences)

,macska,kutya
macska macska macska,3,0
kutya kutya,0,2
kutya macska kutya,1,2


In [19]:

fig = px.scatter(pd.DataFrame(vect, index=sentences).reset_index(), x='macska', y='kutya', hover_data='index')
fig.update_layout(width=800, height=600)
fig.update_xaxes(range=[-0.5, 3.5])
fig.update_yaxes(range=[-0.5, 2.5])
fig.show()

### Nagyobb péda

In [21]:
sentences = [
    "A kutya átfut a parkon.",
    "A macska átfut a parkon.",
    "A macska egeret fogott.",
    "A macska szereti a kutyát, nem az egeret.",
    "Az egeret szereti a macska, nem a kutyát."
]

In [22]:
clean_sentences = [s.replace(".", "").replace(",", "").lower() for s in sentences]
clean_sentences

['a kutya átfut a parkon',
 'a macska átfut a parkon',
 'a macska egeret fogott',
 'a macska szereti a kutyát nem az egeret',
 'az egeret szereti a macska nem a kutyát']

In [23]:
words = [w for s in clean_sentences for w in s.split(" ")]
words

['a',
 'kutya',
 'átfut',
 'a',
 'parkon',
 'a',
 'macska',
 'átfut',
 'a',
 'parkon',
 'a',
 'macska',
 'egeret',
 'fogott',
 'a',
 'macska',
 'szereti',
 'a',
 'kutyát',
 'nem',
 'az',
 'egeret',
 'az',
 'egeret',
 'szereti',
 'a',
 'macska',
 'nem',
 'a',
 'kutyát']

In [5]:
# szógyakoriság
from collections import Counter

c = Counter(words)
c.most_common(5)

[('a', 9), ('macska', 4), ('egeret', 3), ('átfut', 2), ('parkon', 2)]

In [6]:
# nem ismert szavakkal mi a helyzet?
c['zöld']

0

In [7]:
# reprezentáció
dictionary = list(set(words))  # duplikációk eltűntetése

def convert_senctence_to_bow(sentence):
  """
  Lefordítja a mondatot számvektorra szózsák módszer szerint
  """
  c_sentence = sentence.replace(".", "").replace(",", "").lower().split(" ")
  c = Counter(c_sentence)
  bow = {k: c[k] for k in dictionary}
  return bow


convert_senctence_to_bow("A kutya és a macska egeret fogott.")

{'parkon': 0,
 'fogott': 1,
 'egeret': 1,
 'nem': 0,
 'az': 0,
 'kutyát': 0,
 'átfut': 0,
 'a': 2,
 'kutya': 1,
 'szereti': 0,
 'macska': 1}

In [8]:
bow_of_sentences = [convert_senctence_to_bow(s) for s in sentences]

pd.DataFrame(bow_of_sentences, index=sentences)

,parkon,fogott,egeret,nem,az,kutyát,átfut,a,kutya,szereti,macska
A kutya átfut a parkon.,1,0,0,0,0,0,1,2,1,0,0
A macska átfut a parkon.,1,0,0,0,0,0,1,2,0,0,1
A macska egeret fogott.,0,1,1,0,0,0,0,1,0,0,1
"A macska szereti a kutyát, nem az egeret.",0,0,1,1,1,1,0,2,0,1,1
"Az egeret szereti a macska, nem a kutyát.",0,0,1,1,1,1,0,2,0,1,1


In [9]:
# hasonlóság = vektortávolság
def calculate_distances(text1, text2):

  bow1 = convert_senctence_to_bow(text1)
  bow2 = convert_senctence_to_bow(text2)
  vector1 = [bow1[k] for k in dictionary]
  vector2 = [bow2[k] for k in dictionary]

  first = np.array(vector1)
  second = np.array(vector2)

  # euklideszi távolság
  dist = np.linalg.norm(first - second)

  # cosine hasonlóság (két vektor által bezárt szög koszinusza)
  cos_sim = np.dot(first, second ) / (np.linalg.norm(first) * np.linalg.norm(second) + 1e-12)

  return (dist, cos_sim)

In [10]:
dist, cos_sim = calculate_distances(sentences[0], sentences[1])
print(sentences[0])
print(sentences[1])
print(f"Eukklédeszi távolság: {dist}")
print(f"Cosine hasonlóság: {cos_sim}")

A kutya átfut a parkon.
A macska átfut a parkon.
Eukklédeszi távolság: 1.4142135623730951
Cosine hasonlóság: 0.8571428571427345


In [12]:
dist, cos_sim = calculate_distances(sentences[0], sentences[2])
print(sentences[0])
print(sentences[2])
print(f"Eukklédeszi távolság: {dist}")
print(f"Cosine hasonlóság: {cos_sim}")

A kutya átfut a parkon.
A macska egeret fogott.
Eukklédeszi távolság: 2.6457513110645907
Cosine hasonlóság: 0.37796447300915575


In [13]:
# probléma: nincs szórend vagy kontextus
dist, cos_sim = calculate_distances(
    "A macska szereti a kutyát, nem az egeret.",
    "A macska szereti az egeret, nem a kutyát."
  )
print(f"Eukklédeszi távolság: {dist}")
print(f"Cosine hasonlóság: {cos_sim}")

Eukklédeszi távolság: 0.0
Cosine hasonlóság: 0.9999999999998999


In [14]:
# probláma: csak azokat a szavakat ismeri, amikre megtanítottuk, minden más nullvektor => távolság = 1, cosine = 0
dist, cos_sim = calculate_distances("Tegnap havazott.", "A Himalája nagyon magas.")
print(f"Eukklédeszi távolság: {dist}")
print(f"Cosine hasonlóság: {cos_sim}")

Eukklédeszi távolság: 1.0
Cosine hasonlóság: 0.0


## Szóbeágyazások (word embeddings)

In [2]:
!pip install gensim

In [3]:
import gensim.downloader

In [4]:
%%time

wiki_vectors = gensim.downloader.load('glove-wiki-gigaword-50')

[==================================================] 100.0% 66.0/66.0MB downloaded
CPU times: user 26 s, sys: 1.56 s, total: 27.5 s
Wall time: 32.4 s


In [5]:
# Szóreprezentáció

wiki_vectors['apple']

array([ 0.52042 , -0.8314  ,  0.49961 ,  1.2893  ,  0.1151  ,  0.057521,
       -1.3753  , -0.97313 ,  0.18346 ,  0.47672 , -0.15112 ,  0.35532 ,
        0.25912 , -0.77857 ,  0.52181 ,  0.47695 , -1.4251  ,  0.858   ,
        0.59821 , -1.0903  ,  0.33574 , -0.60891 ,  0.41742 ,  0.21569 ,
       -0.07417 , -0.5822  , -0.4502  ,  0.17253 ,  0.16448 , -0.38413 ,
        2.3283  , -0.66682 , -0.58181 ,  0.74389 ,  0.095015, -0.47865 ,
       -0.84591 ,  0.38704 ,  0.23693 , -1.5523  ,  0.64802 , -0.16521 ,
       -1.4719  , -0.16224 ,  0.79857 ,  0.97391 ,  0.40027 , -0.21912 ,
       -0.30938 ,  0.26581 ], dtype=float32)

In [ ]:
# Hány szó van a modellben?

f"{len(wiki_vectors):,}"

'400,000'

In [ ]:
# Hol van az "apple"?

wiki_vectors.key_to_index['apple']

3292

In [ ]:
np.random.seed = 2234

In [ ]:
# mik vannak benne? - 25 véletlen példa
np.random.choice(wiki_vectors.index_to_key[3000:5000], 25, replace=False)

array(['trillion', 'quarterback', 'stages', 'par', 'economist', 'files',
       'counsel', 'lowest', 'isolated', 'touch', 'slowly', 'fit',
       'welfare', 'survivors', 'fundamental', '1952', 'campbell',
       'reflect', 'motors', 'campaigns', 'nobel', 'smoking', 'passes',
       'taipei', 'uh'], dtype='<U18')

In [6]:
# ezt a modellt a 2010-es évek közepén tanították fel, úgyogy a kifejezések amik azőta születtek nincsenek benne
wiki_vectors['tiktok']

KeyError: "Key 'tiktok' not present"

### Szavak hasonlósága - szinonímák

In [7]:
similar_to_smile = wiki_vectors.most_similar(['smile'])
similar_to_smile

[('grin', 0.8619306683540344),
 ('smiles', 0.8397865295410156),
 ('eyes', 0.7954673767089844),
 ('smiling', 0.7861469388008118),
 ('laugh', 0.7707611322402954),
 ('touch', 0.7635201811790466),
 ('hug', 0.7596133351325989),
 ('bit', 0.7371944189071655),
 ('gentle', 0.7313948273658752),
 ('smirk', 0.7309232354164124)]

In [11]:
from sklearn.manifold import TSNE

def draw_plot(words):
  tsne = TSNE(n_components=2, random_state=0, perplexity=1)
  vectors = np.array([wiki_vectors[word] for word in words])
  Y = tsne.fit_transform(vectors)

  x_data, y_data = zip(*Y)
  fig = px.scatter(x=x_data, y=y_data, hover_name=words)
  fig.show()




In [14]:
# words = ["king", "queen", "computer", "mouse", "joy", "sadness"]
words = ["king", "queen", "princess", "prince"]

draw_plot(words)


## Fontos, hogy milyen adatokon tanítunk

In [15]:
%%time

twitter_vectors = gensim.downloader.load('glove-twitter-50')

[==================================================] 100.0% 199.5/199.5MB downloaded
CPU times: user 1min 18s, sys: 4.88 s, total: 1min 23s
Wall time: 1min 38s


In [16]:
similar_to_smile

[('grin', 0.8619306683540344),
 ('smiles', 0.8397865295410156),
 ('eyes', 0.7954673767089844),
 ('smiling', 0.7861469388008118),
 ('laugh', 0.7707611322402954),
 ('touch', 0.7635201811790466),
 ('hug', 0.7596133351325989),
 ('bit', 0.7371944189071655),
 ('gentle', 0.7313948273658752),
 ('smirk', 0.7309232354164124)]

In [17]:
twitter_vectors.most_similar(['smile'])

[('kiss', 0.8498159050941467),
 ('heart', 0.8433985710144043),
 ('eyes', 0.8414220809936523),
 ('hug', 0.8323259353637695),
 ('love', 0.827944278717041),
 ('smiling', 0.8259447813034058),
 ('you', 0.8204987645149231),
 ('always', 0.8151404857635498),
 ('hugs', 0.8127555847167969),
 ('your', 0.801873505115509)]

In [ ]:
wiki_vectors.most_similar(['arms'])

[('weapons', 0.8018144369125366),
 ('hand', 0.7318101525306702),
 ('armed', 0.7299286127090454),
 ('carrying', 0.724751353263855),
 ('carry', 0.7231833338737488),
 ('cross', 0.7169591784477234),
 ('pressed', 0.7155767679214478),
 ('holding', 0.7145681977272034),
 ('military', 0.7131772041320801),
 ('crack', 0.7077354192733765)]

In [ ]:
twitter_vectors.most_similar(['arms'])

[('hands', 0.886970043182373),
 ('neck', 0.8406134843826294),
 ('arm', 0.8387513160705566),
 ('chest', 0.8353148102760315),
 ('feet', 0.8316022753715515),
 ('legs', 0.8287435173988342),
 ('knees', 0.8170155882835388),
 ('hand', 0.8160897493362427),
 ('shoulders', 0.8130438327789307),
 ('fingers', 0.812550961971283)]